In [ ]:
%gui qt
%matplotlib qt
from glob import glob
import os, sys
# only necessary if using without installing
sys.path.append("..")
from xmcd_projection import *
from skimage.io import imread, imsave
from PIL import Image
import meshio
import trimesh

In [97]:
import importlib
import xmcd_projection
importlib.reload(xmcd_projection)
from xmcd_projection import *

### Get file paths

In [ ]:
msh_file = "mumax_mesh.vtu"
mag_file = "mumax_mag.csv"
# scale for the points
scale = 1e9

## Generate raytracing - skip if generated

In [ ]:
# get the mesh, scale the points to nm
msh = Mesh.from_file(msh_file, scale=scale)

#### Make sure that the projection vector is correct and that the structure is oriented well

In [33]:
# get the projection vector
p = get_projection_vector(90, 0) # direction of xrays
n = [0, 1, 0] # normal to the projection plane
x0 = [-100, 0, 0] # point on the projection plane

# prepare raytracing object
raytr = RayTracing(msh, p, n=n, x0=x0)
struct = raytr.struct
struct_projected = raytr.struct_projected

In [ ]:
vis = MeshVisualizer(struct, struct_projected)
vis.set_camera(dist=2e5)
vis.show()

## If raytracing file generated - skip if not

In [ ]:
# load raytracing if exists
raytr = np.load("raytracing.npy", allow_pickle=True).item()
struct = raytr.struct
struct_projected = raytr.struct_projected

## Generate and save raytracing

In [44]:

from trimesh import triangles as triangles_mod
from xmcd_projection.raytracing import *

struct_projected = raytr.struct_projected
triangles = struct_projected.triangles
tol = 1e-3

triangles_normal = triangles_mod.normals(triangles)[0]
tree = triangles_mod.bounds_tree(triangles)

pnew = p[np.newaxis, :]
ray_origins = points

def ray_piercing_fun(orig): return ray_triangle_id(
    triangles, orig[np.newaxis, :], pnew, triangles_normal=triangles_normal, tree=tree)

def get_piercings_item(orig):
    tri_id, _, locs = ray_piercing_fun(orig)
    try:
        return get_piercings_frompt_lengths(locs, tri_id // 4)
    except ValueError as e:
        warn(
            str(e) + ' If this happens rarely, it could be a numerical artefact.')
        # running this again, if it crashes a second time, it's not an artefact!
        tri_id, _, locs = ray_piercing_fun(orig + tol)
        return get_piercings_frompt_lengths(locs, tri_id // 4)

for orig in ray_origins:
    get_piercings_item(orig)
piercings_list = [get_piercings_item(orig) for orig in ray_origins]

<ipython-input-44-27d5fb53fdf3>:22: UserWarning: Wrong number of intersections! Ensure that tetrahedra are valid and that the projection plane does not intersect the structure. If this happens rarely, it could be a numerical artefact.
  warn(


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(float64, 1d, C), Tuple(Literal[int](0), slice<a:b>))
 
There are 22 candidate implementations:
[1m   - Of which 20 did not match due to:
   Overload of function 'getitem': File: <numerous>: Line N/A.
     With argument(s): '(array(float64, 1d, C), Tuple(int64, slice<a:b>))':[0m
[1m    No match.[0m
[1m   - Of which 1 did not match due to:
   Overload in function 'GetItemBuffer.generic': File: numba\core\typing\arraydecl.py: Line 162.
     With argument(s): '(array(float64, 1d, C), Tuple(int64, slice<a:b>))':[0m
[1m    Rejected as the implementation raised a specific error:
      TypeError: cannot index array(float64, 1d, C) with 2 indices: Tuple(int64, slice<a:b>)[0m
  raised from c:\users\lukas\miniconda3\lib\site-packages\numba\core\typing\arraydecl.py:84
[1m   - Of which 1 did not mat

In [56]:

tri_id, _, locs = ray_piercing_fun(orig+tol)

In [61]:
triangles.shape


(3188, 3, 3)

In [62]:
triangles = struct_projected.triangles

In [63]:
triangles[0,:,:]

array([[35.        ,  0.        ,  5.        ],
       [45.        ,  0.        ,  5.        ],
       [35.        ,  0.        , 14.99999905]])

In [94]:
a = np.linalg.norm(triangles[:, 1, :] - triangles[:, 2, :], axis=1)[:, None]
b = np.linalg.norm(triangles[:, 2, :] - triangles[:, 0, :], axis=1)[:, None]
c = np.linalg.norm(triangles[:, 1, :] - triangles[:, 0, :], axis=1)[:, None]
triangle_centres = (a*triangles[:, 0, :]+b*triangles[:, 1, :]+c*triangles[:, 2, :])/(a+b+c)

In [95]:
i = 12
import matplotlib.pyplot as plt
plt.plot(triangles[i, :, 2], triangles[i, :, 0])
plt.scatter(triangle_centres[i, 2], triangle_centres[i, 0])
plt.axis('equal')

(4.500000047683716, 15.499998998641967, 44.5, 55.5)

In [ ]:
get_piercings_frompt_lengths(locs, tri_id // 4)

In [ ]:
np.max(tri_areas)

In [34]:
self = raytr
points = self.struct_projected.triangles_center
self._piercings = raytracing.get_points_piercings(
    points, self.p, self.mesh.triangles, tol=self.tol)




  0%|                                                                                         | 0/3188 [00:00<?, ?it/s]


  4%|██▉                                                                          | 123/3188 [00:00<00:02, 1219.16it/s]

ValueError: Wrong number of intersections! Ensure that tetrahedra are valid and that the projection plane does not intersect the structure.




  4%|██▉                                                                          | 123/3188 [00:19<00:02, 1219.16it/s]

In [105]:
import xmcd_projection
deep_reload(xmcd_projection)
from xmcd_projection import *

In [106]:
raytr.get_piercings()
# np.save("raytracing.npy", raytr, allow_pickle=True)




  0%|                                                                                         | 0/3188 [00:00<?, ?it/s]


  0%|                                                                                 | 1/3188 [00:01<58:38,  1.10s/it]


  2%|█▉                                                                              | 78/3188 [00:01<40:04,  1.29it/s]


  5%|███▌                                                                           | 145/3188 [00:01<27:28,  1.85it/s]


  6%|█████▏                                                                         | 207/3188 [00:01<18:51,  2.63it/s]


  9%|███████▍                                                                       | 302/3188 [00:01<12:47,  3.76it/s]


 12%|█████████▌                                                                     | 388/3188 [00:01<08:42,  5.36it/s]


 15%|████████████▏                                                                  | 493/3188 [00:01<05:52,  7.64it/s]


 19%|██████████████▌ 

## Get the xmcd

#### Get magnetisation, fix vertex shuffling
Note: sometimes if the mesh file has multiple parts, the paraview export and the original mesh coordinates are not in the same order. I add a function to fix that when necessary

In [ ]:
magnetisation, mag_points = load_mesh_magnetisation(mag_file, scale=scale)
shuffle_file = "shuffle_indx.npy"
try:
    shuffle_indx = np.load(shuffle_file)
except FileNotFoundError:
    print('File not found. Generating shuffle indx')
    shuffle_indx = msh.get_shuffle_indx(mag_points)
    np.save(shuffle_file, shuffle_indx)
magnetisation = magnetisation[shuffle_indx, :]

### Get the colours and XMCD values

In [ ]:
xmcd_value = raytr.get_xmcd(magnetisation)
mag_colors = get_struct_face_mag_color(struct, magnetisation)

In [ ]:
azi=90
center_struct = [0, 0, 0]
dist_struct = 1e4
center_peem = [100, -200, 0]
dist_peem = 8e4

vis = MeshVisualizer(struct, struct_projected, projected_xmcd=xmcd_value, struct_colors=mag_colors)
vis.show(azi=azi, center=center_peem, dist=dist_peem)
# Image.fromarray(vis.get_image_np())

#### View different parts of the image separately

#### Both

In [ ]:

vis.update_colors(xmcd_value, mag_colors)
vis.view_both(azi=azi, center=center_peem, dist=dist_peem)
Image.fromarray(vis.get_image_np())

#### Projection

In [ ]:
vis.view_projection(azi=azi, center=center_peem, dist=dist_peem)
Image.fromarray(vis.get_image_np())

#### Structure

In [ ]:

center_struct = [75, 50, 0]
dist_struct = 1e4
vis.view_struct(azi=azi, center=center_struct, dist=dist_struct)
Image.fromarray(vis.get_image_np())

#### Blurred image

In [ ]:
vis.view_projection(azi=azi, center=center_peem, dist=dist_peem)
Image.fromarray((vis.get_blurred_image(desired_background=0.7)*255).astype(np.uint8))

#### Saving one render

In [ ]:
vis.view_both(azi=azi, center=center_peem, dist=dist_peem)
vis.save_render('mumax_shadow.png')
vis.view_projection()
blurred = vis.get_blurred_image(desired_background=0.7)
imsave('mumax_shadow_blurred.png', (blurred*255).astype(np.uint8), check_contrast=False)

vis.view_struct(azi=azi, center=center_struct, dist=dist_struct)
vis.save_render('mumax_structure_view.png')